# 📕 MODULE 2.3 : Manipulation Avancée des PDF

**Objectif du TP :** Apprendre à manipuler des publications académiques réelles au format PDF.

Nous allons utiliser la bibliothèque **PyPDF2** pour :
1.  Lire des articles scientifiques.
2.  Extraire leur texte brut pour l'analyser (NLP).
3.  Fusionner plusieurs articles en un seul fichier.

---

## 🛠️ ÉTAPE 1 : Installation et Téléchargement des Données

Nous commençons par installer `PyPDF2` et télécharger vos deux articles de recherche.

In [ ]:
# 1. Installation de la librairie
!pip install PyPDF2

# 2. Téléchargement des articles (Cas réel)
# Article 1 : Engineering Education (i-jep)
!wget -O article1.pdf "https://online-journals.org/index.php/i-jep/article/download/40853/14311/140647"

# Article 2 : Educational Research (ERIC)
!wget -O article2.pdf "https://files.eric.ed.gov/fulltext/EJ1481879.pdf"

print("✅ Les fichiers 'article1.pdf' et 'article2.pdf' sont prêts !")

## 📖 ÉTAPE 2 : Lecture et Métadonnées

Commençons par analyser le premier article (`article1.pdf`).

**Note Pédagogique :**
Un fichier PDF est un fichier **binaire**. Il faut toujours utiliser le mode `'rb'` (**R**ead **B**inary) dans la fonction `open()`.

In [ ]:
import PyPDF2

# 1. Ouverture du fichier en mode binaire
f = open('article1.pdf', 'rb')

# 2. Création du Lecteur (Reader)
# C'est l'objet qui va décoder le format PDF
pdf_reader = PyPDF2.PdfReader(f)

# 3. Combien de pages ?
# (Anciennement .numPages, maintenant len(pages))
nb_pages = len(pdf_reader.pages)

print(f"L'article 1 contient {nb_pages} pages.")
print(f"Métadonnées : {pdf_reader.metadata}")

## 📝 ÉTAPE 3 : Extraction du Texte (Page par Page)

Contrairement à un fichier Word, un PDF n'est pas du texte continu. C'est une suite de pages graphiques.
Nous devons extraire le texte de chaque page individuellement.

In [ ]:
# Récupérons la première page (Index 0)
page_une = pdf_reader.pages[0]

# Extraction du texte
texte_page_1 = page_une.extract_text()

print("--- DÉBUT DE L'ARTICLE ---")
print(texte_page_1[:1000]) # On affiche les 1000 premiers caractères
print("...")

### 🧠 Analyse NLP rapide
Maintenant que nous avons le texte dans une variable Python (`texte_page_1`), nous pouvons l'analyser comme n'importe quelle chaîne de caractères (compter les mots, chercher des entités...).

In [ ]:
# Exemple : Découpage en mots (Tokenization basique)
mots = texte_page_1.split()
print(f"La première page contient environ {len(mots)} mots.")
print(f"Les 10 premiers mots : {mots[:10]}")

## 🔄 ÉTAPE 4 : Extraction Totale (Boucle)

Pour analyser tout l'article, nous devons boucler sur toutes les pages et concaténer le texte.

In [ ]:
texte_complet = []

for i in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[i]
    texte_complet.append(page.extract_text())

# On ferme le fichier une fois fini
f.close()

print(f"Extraction terminée pour {len(texte_complet)} pages.")

## 🧱 ÉTAPE 5 : Fusion de PDF (Merger)

Imaginez que vous voulez créer un **Dossier de Recherche** unique contenant vos deux articles (`article1.pdf` et `article2.pdf`).
Nous utilisons `PdfMerger`.

In [ ]:
from PyPDF2 import PdfMerger

# Création du fusionneur
merger = PdfMerger()

# Liste des fichiers à fusionner
pdfs_a_fusionner = ['article1.pdf', 'article2.pdf']

for pdf in pdfs_a_fusionner:
    print(f"Ajout de {pdf}...")
    # On ouvre chaque fichier et on l'ajoute à la pile
    merger.append(open(pdf, 'rb'))

# Écriture du résultat final
with open("Dossier_Recherche_Complet.pdf", "wb") as fout:
    merger.write(fout)

print("✅ Fusion terminée ! Le fichier 'Dossier_Recherche_Complet.pdf' est créé.")

## ✂️ ÉTAPE 6 : Découpage (Créer un PDF extrait)

Parfois, vous voulez extraire **seulement la première page** (le résumé/abstract) d'un article pour l'envoyer à un collègue.

In [ ]:
# 1. Lecture de l'article source
f_source = open('article1.pdf', 'rb')
reader = PyPDF2.PdfReader(f_source)

# 2. Création d'un écrivain vide
writer = PyPDF2.PdfWriter()

# 3. On prend seulement la page 0 (la première)
writer.add_page(reader.pages[0])

# 4. Sauvegarde dans un nouveau fichier
with open("Resume_Article1.pdf", "wb") as f_out:
    writer.write(f_out)

f_source.close()
print("✅ Fichier 'Resume_Article1.pdf' créé (1 page).")